##Импорты


In [ ]:
!pip install wget
!pip install gensim
!pip install ufal.udpipe
!pip install datasets
!pip install torch

In [ ]:
#импорт библиотек
import torch
import torchvision
from torchvision import transforms, datasets
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.nn.functional as F
import scipy
import torch.utils.data as data_utils
from sklearn.metrics import classification_report
import pandas as pd
import torch.optim as optim
import numpy as np
from sklearn.metrics import f1_score
import re
import nltk # импорт библиотеки NLTK
from nltk.tokenize import word_tokenize # импорт функции для токенизации
# загрузка списков стоп слов
nltk.download('stopwords')
nltk.download('punkt')
from pymystem3 import Mystem
import sys
import os
import wget
import re
from ufal.udpipe import Model, Pipeline
import gensim
from gensim.models import Word2Vec
import sys
import logging
import datasets

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##Загрузка данных

In [ ]:
dataset_with_tags = datasets.load_from_disk("/content/drive/MyDrive/df_with_tags")
tags = [row['tags'] for row in dataset_with_tags]
labels = [row['labels'] for row in dataset_with_tags]

In [ ]:
dataset_without_tags = datasets.load_from_disk("/content/drive/MyDrive/df_without_tags")
no_tags = [row['no_tags'] for row in dataset_without_tags]
labels_nt = [row['labels'] for row in dataset_without_tags]

In [ ]:
def word_averaging(model, words):
    all_words, mean = set(), []

    for word in words:
        if word in model.key_to_index.keys():
            mean.append(model[model.key_to_index[word]])
            all_words.add(model.key_to_index[word])

    if not mean:
        #logging.warning("cannot compute similarity with no input %s", words)
        # FIXME: remove these examples in pre-processing
        return np.zeros(model.vector_size)

    mean = gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)
    return mean

In [ ]:
def  word_averaging_list(model, text_list):
    return np.vstack([word_averaging(model, comment_text) for comment_text in text_list ])

##Загрузка моделей

###Word2vec

In [ ]:
w2v_model = Word2Vec(sentences=no_tags, vector_size=300, window=5, min_count=1, workers=4)

In [ ]:
w2v_model.train(no_tags, total_examples=w2v_model.corpus_count, epochs=300, report_delay=1)
w2v_model.init_sims(replace=True)

In [ ]:
w2v_model_X = word_averaging_list(w2v_model.wv, no_tags)
y = labels

In [ ]:
w2v_model.wv.save_word2vec_format('my_w2v.vectors', binary=False)

In [ ]:
from gensim.models import KeyedVectors


In [ ]:
my_w2v_vectors = KeyedVectors.load_word2vec_format('/content/my_w2v.vectors', binary=False)

In [ ]:
w2v_model.wv

In [ ]:
w2v_model_X.shape

(22398, 300)

###Pretrained word2vec

In [ ]:
pretrained_w2v = gensim.models.KeyedVectors.load_word2vec_format('/content/drive/MyDrive/180/model.bin', encoding='utf-8', unicode_errors='ignore', binary=True)
pretrained_w2v.init_sims(replace=True)

In [ ]:
pretrained_w2v_X = word_averaging_list(pretrained_w2v, tags)
y = labels
pretrained_w2v_X.shape

(22398, 300)

###FastText

In [ ]:
fasttext_model = gensim.models.FastText(sentences=no_tags, vector_size=300, window=5, min_count=1, workers=4)

In [ ]:
fasttext_model.train(no_tags, total_examples=fasttext_model.corpus_count, epochs=300, report_delay=1)
fasttext_model.build_vocab(no_tags)
fasttext_model.init_sims(replace=True)

In [ ]:
fasttext_model_X = word_averaging_list(fasttext_model.wv, no_tags)
y = labels_nt

In [ ]:
fasttext_model_X.shape

(22398, 300)

###FastText pretrained

In [ ]:
pretrained_ft = gensim.models.KeyedVectors.load('/content/drive/MyDrive/213/model.model')
pretrained_ft.init_sims(replace=True)

In [ ]:
pretrained_ft_X = word_averaging_list(pretrained_ft, no_tags)
y = labels_nt

In [ ]:
pretrained_ft_X.shape

(22398, 300)

#Обучение

In [ ]:
import itertools

In [ ]:
activations = [F.leaky_relu, F.relu]
optimizers = [optim.Adam, optim.SGD]
weight_inits = [nn.init.kaiming_uniform_, nn.init.xavier_uniform_]
dropouts=[True, False]
L2s=[True, False]
normalizations = [nn.BatchNorm1d, nn.LayerNorm]
schedulers = [optim.lr_scheduler.ExponentialLR, optim.lr_scheduler.MultiStepLR]

params = [activatios, optimizers, weight_inits, dropouts, L2s normalizations, schedulers]

all_combinations = list(product(*params))

In [ ]:
num_layers = 3
num_neurons = [[300, 128], [128, 128], [128, 2]]

In [ ]:
class Net(nn.Module):
    def __init__(self, num_layers, num_neurons, activation, weight_init, drop_out, normalization):
        super(Net, self).__init__()

        self.weight_init = weight_init
        self.activation = activation

        self.layers = nn.ModuleList()
        self.lns = nn.ModuleList()

        for i in range(num_layers):
            in_features, out_features = num_neurons[i]
            self.layers.append(nn.Linear(in_features, out_features))
            self.lns.append(normalization(out_features))

            # Инициализация весов
            self.weight_init(self.layers[i].weight)

        self.dropout = nn.Dropout(drop_out)

    def forward(self, x):
        for i in range(len(self.layers)):
            x = self.lns[i](self.activation(self.layers[i](x)))
            x = self.dropout(x)

        return x

In [ ]:
def split(X, y, batch_size):
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
  #преобразование в тензоры
  inputs_train = torch.tensor(X_train)
  targets_train = torch.tensor(y_train)

  inputs_test = torch.tensor(X_test)
  targets_test = torch.tensor(y_test)

  train = data_utils.TensorDataset(inputs_train, targets_train)
  test = data_utils.TensorDataset(inputs_test, targets_test)

  trainset = torch.utils.data.DataLoader(train, shuffle=True, batch_size=batch_size, drop_last=True)
  testset = torch.utils.data.DataLoader(test, shuffle=False, batch_size=batch_size, drop_last=True)

  return trainset, testset, y_test

In [ ]:
def train_model(trainset, net, optimizer, scheduler, L2):
    weight_decay = 0.01 if L2 else 0
    opt = optimizer(net.parameters(), lr=0.1, weight_decay=weight_decay)
    if scheduler.__name__ == 'MultiStepLR':
        sched = scheduler(opt, milestones=[2, 4], gamma=0.1)
    else:
        sched = scheduler(opt, gamma=0.1)

    losses = []

    for epoch in range(5):
        running_loss = 0.0
        for i, data in enumerate(trainset, 0):
            inputs, labels = data
            opt.zero_grad()  # устанавливаем значение градиента в 0 перед вычислением функции потерь
            outputs = net(inputs.float())
            loss = nn.NLLLoss()(torch.log_softmax(outputs, dim=1), labels.long())  # вычисляем функцию потерь
            loss.backward()  # передаем это значение назад по сети
            opt.step()  # пытаемся оптимизировать значение весов исходя из потерь и градиента

            # Выводим лосс
            running_loss += loss.item()
            if i % 1000 == 999:  # печатаем каждые 1000 мини-батчей
                print('[%d, %5d] loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / 1000))
                running_loss = 0.0

        # Сохраняем значение loss для текущей эпохи
        losses.append(running_loss)

        # Выводим сообщение об окончании эпохи
        print('Epoch %d finished' % (epoch + 1))

        # Планировщик шагов
        sched.step()

    print('Finished Training')

    return loss.item(), net

In [ ]:
def weighted_f1(net, testset):
    net.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for data in testset:
            inputs, labels = data
            outputs = net(inputs)
            _, predicted = torch.max(outputs, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    all_labels = np.array(all_labels)
    all_preds = np.array(all_preds)

    all_labels_binary = (all_labels > 0).astype(int)
    all_preds_binary = (all_preds > 0).astype(int)

    weighted_f1 = f1_score(all_labels_binary, all_preds_binary, average='weighted')
    return weighted_f1

In [ ]:
column_names = ['Activation', 'Optimizer', 'Weight Initialization', 'Regularization', 'Dropout', 'Normalization', 'Scheduler', 'F1']

In [ ]:
import more_itertools as mit

In [ ]:
def train(params, X, y):
    res_list = []
    for params in parameters:
        drop_out = 0.5 if dropout else 0
        batch_size = 32 if normalization.__name__=='BatchNorm1d' else  1
        trainset, testset, y_test = split(X, y, batch_size)
        net = Net(num_layers, num_neurons, activation, weight_init, drop_out, normalization)
        loss, net, weighted_f1_score = train_model(trainset, net, optimizer, scheduler, L2, testset)


        res_df = pd.DataFrame([{
            'Activation': activation,
            'Optimizer': optimizer,
            'Weight Initialization': weight_init,
            'Dropout': drop_out,
            'Regularization': L2,
            'Normalization': normalization,
            'Scheduler': scheduler,
            'F1': weighted_f1_score
        }])


        res_list.append(res_df)


    final_res_df = pd.concat(res_list, ignore_index=True)
    return final_res_df

In [ ]:
w2v_model_res = pd.DataFrame(columns=column_names)
pretrained_w2v_res = pd.DataFrame(columns=column_names)
fasttext_model_res = pd.DataFrame(columns=column_names)
pretrained_ft_res = pd.DataFrame(columns=column_names)

In [ ]:
w2v_model_res_list = []
w2v_model_res = train(all_combinations, w2v_model_X, y)

In [ ]:
w2v_model_res.sort_values(by='F1', ascending=False)

,Activation,Optimizer,Weight Initialization,Dropout,Regularization,Normalization,Scheduler,F1
105,<function relu at 0x7e268c6eed40>,<class 'torch.optim.sgd.SGD'>,<function kaiming_uniform_ at 0x7e268c56f130>,0.0,True,<class 'torch.nn.modules.batchnorm.BatchNorm1d'>,<class 'torch.optim.lr_scheduler.MultiStepLR'>,0.698876
121,<function relu at 0x7e268c6eed40>,<class 'torch.optim.sgd.SGD'>,<function xavier_uniform_ at 0x7e268c56ef80>,0.0,True,<class 'torch.nn.modules.batchnorm.BatchNorm1d'>,<class 'torch.optim.lr_scheduler.MultiStepLR'>,0.697294
109,<function relu at 0x7e268c6eed40>,<class 'torch.optim.sgd.SGD'>,<function kaiming_uniform_ at 0x7e268c56f130>,0.0,False,<class 'torch.nn.modules.batchnorm.BatchNorm1d'>,<class 'torch.optim.lr_scheduler.MultiStepLR'>,0.696873
41,<function leaky_relu at 0x7e268c6ef130>,<class 'torch.optim.sgd.SGD'>,<function kaiming_uniform_ at 0x7e268c56f130>,0.0,True,<class 'torch.nn.modules.batchnorm.BatchNorm1d'>,<class 'torch.optim.lr_scheduler.MultiStepLR'>,0.696493
56,<function leaky_relu at 0x7e268c6ef130>,<class 'torch.optim.sgd.SGD'>,<function xavier_uniform_ at 0x7e268c56ef80>,0.0,True,<class 'torch.nn.modules.batchnorm.BatchNorm1d'>,<class 'torch.optim.lr_scheduler.ExponentialLR'>,0.696422
...,...,...,...,...,...,...,...,...
95,<function relu at 0x7e268c6eed40>,<class 'torch.optim.adam.Adam'>,<function xavier_uniform_ at 0x7e268c56ef80>,0.0,False,<class 'torch.nn.modules.normalization.LayerNo...,<class 'torch.optim.lr_scheduler.MultiStepLR'>,0.364703
31,<function leaky_relu at 0x7e268c6ef130>,<class 'torch.optim.adam.Adam'>,<function xavier_uniform_ at 0x7e268c56ef80>,0.0,False,<class 'torch.nn.modules.normalization.LayerNo...,<class 'torch.optim.lr_scheduler.MultiStepLR'>,0.364703
30,<function leaky_relu at 0x7e268c6ef130>,<class 'torch.optim.adam.Adam'>,<function xavier_uniform_ at 0x7e268c56ef80>,0.0,False,<class 'torch.nn.modules.normalization.LayerNo...,<class 'torch.optim.lr_scheduler.ExponentialLR'>,0.364703
27,<function leaky_relu at 0x7e268c6ef130>,<class 'torch.optim.adam.Adam'>,<function xavier_uniform_ at 0x7e268c56ef80>,0.0,True,<class 'torch.nn.modules.normalization.LayerNo...,<class 'torch.optim.lr_scheduler.MultiStepLR'>,0.364703


In [ ]:
pretrained_w2v_res_list = []

In [ ]:
pretrained_w2v_res = train(all_combinations, pretrained_w2v_X, y)

In [ ]:
pretrained_w2v_res.sort_values(by='F1', ascending=False)

,Activation,Optimizer,Weight Initialization,Dropout,Regularization,Normalization,Scheduler,F1
114,<function relu at 0x7c8105bb2ef0>,<class 'torch.optim.sgd.SGD'>,<function xavier_uniform_ at 0x7c80f5b33130>,0.5,True,<class 'torch.nn.modules.normalization.LayerNo...,<class 'torch.optim.lr_scheduler.ExponentialLR'>,0.504500
3,<function leaky_relu at 0x7c8105bb32e0>,<class 'torch.optim.adam.Adam'>,<function kaiming_uniform_ at 0x7c80f5b332e0>,0.5,True,<class 'torch.nn.modules.normalization.LayerNo...,<class 'torch.optim.lr_scheduler.MultiStepLR'>,0.497359
66,<function relu at 0x7c8105bb2ef0>,<class 'torch.optim.adam.Adam'>,<function kaiming_uniform_ at 0x7c80f5b332e0>,0.5,True,<class 'torch.nn.modules.normalization.LayerNo...,<class 'torch.optim.lr_scheduler.ExponentialLR'>,0.496870
48,<function leaky_relu at 0x7c8105bb32e0>,<class 'torch.optim.sgd.SGD'>,<function xavier_uniform_ at 0x7c80f5b33130>,0.5,True,<class 'torch.nn.modules.batchnorm.BatchNorm1d'>,<class 'torch.optim.lr_scheduler.ExponentialLR'>,0.496061
19,<function leaky_relu at 0x7c8105bb32e0>,<class 'torch.optim.adam.Adam'>,<function xavier_uniform_ at 0x7c80f5b33130>,0.5,True,<class 'torch.nn.modules.normalization.LayerNo...,<class 'torch.optim.lr_scheduler.MultiStepLR'>,0.495869
...,...,...,...,...,...,...,...,...
94,<function relu at 0x7c8105bb2ef0>,<class 'torch.optim.adam.Adam'>,<function xavier_uniform_ at 0x7c80f5b33130>,0.0,False,<class 'torch.nn.modules.normalization.LayerNo...,<class 'torch.optim.lr_scheduler.ExponentialLR'>,0.373926
95,<function relu at 0x7c8105bb2ef0>,<class 'torch.optim.adam.Adam'>,<function xavier_uniform_ at 0x7c80f5b33130>,0.0,False,<class 'torch.nn.modules.normalization.LayerNo...,<class 'torch.optim.lr_scheduler.MultiStepLR'>,0.373926
27,<function leaky_relu at 0x7c8105bb32e0>,<class 'torch.optim.adam.Adam'>,<function xavier_uniform_ at 0x7c80f5b33130>,0.0,True,<class 'torch.nn.modules.normalization.LayerNo...,<class 'torch.optim.lr_scheduler.MultiStepLR'>,0.373926
26,<function leaky_relu at 0x7c8105bb32e0>,<class 'torch.optim.adam.Adam'>,<function xavier_uniform_ at 0x7c80f5b33130>,0.0,True,<class 'torch.nn.modules.normalization.LayerNo...,<class 'torch.optim.lr_scheduler.ExponentialLR'>,0.373926


In [ ]:
fasttext_model_res_list = []

In [ ]:
fasttext_model_res = train(all_combinations, fasttext_model_X, y)

In [ ]:
fasttext_model_res.sort_values(by='F1', ascending=False)

,Activation,Optimizer,Weight Initialization,Dropout,Regularization,Normalization,Scheduler,F1
121,<function relu at 0x7efb2b1beef0>,<class 'torch.optim.sgd.SGD'>,<function xavier_uniform_ at 0x7efb1d13f130>,0.0,True,<class 'torch.nn.modules.batchnorm.BatchNorm1d'>,<class 'torch.optim.lr_scheduler.MultiStepLR'>,0.573525
93,<function relu at 0x7efb2b1beef0>,<class 'torch.optim.adam.Adam'>,<function xavier_uniform_ at 0x7efb1d13f130>,0.0,False,<class 'torch.nn.modules.batchnorm.BatchNorm1d'>,<class 'torch.optim.lr_scheduler.MultiStepLR'>,0.572739
57,<function leaky_relu at 0x7efb2b1bf2e0>,<class 'torch.optim.sgd.SGD'>,<function xavier_uniform_ at 0x7efb1d13f130>,0.0,True,<class 'torch.nn.modules.batchnorm.BatchNorm1d'>,<class 'torch.optim.lr_scheduler.MultiStepLR'>,0.572408
13,<function leaky_relu at 0x7efb2b1bf2e0>,<class 'torch.optim.adam.Adam'>,<function kaiming_uniform_ at 0x7efb1d13f2e0>,0.0,False,<class 'torch.nn.modules.batchnorm.BatchNorm1d'>,<class 'torch.optim.lr_scheduler.MultiStepLR'>,0.570352
29,<function leaky_relu at 0x7efb2b1bf2e0>,<class 'torch.optim.adam.Adam'>,<function xavier_uniform_ at 0x7efb1d13f130>,0.0,False,<class 'torch.nn.modules.batchnorm.BatchNorm1d'>,<class 'torch.optim.lr_scheduler.MultiStepLR'>,0.569686
...,...,...,...,...,...,...,...,...
78,<function relu at 0x7efb2b1beef0>,<class 'torch.optim.adam.Adam'>,<function kaiming_uniform_ at 0x7efb1d13f2e0>,0.0,False,<class 'torch.nn.modules.normalization.LayerNo...,<class 'torch.optim.lr_scheduler.ExponentialLR'>,0.371870
79,<function relu at 0x7efb2b1beef0>,<class 'torch.optim.adam.Adam'>,<function kaiming_uniform_ at 0x7efb1d13f2e0>,0.0,False,<class 'torch.nn.modules.normalization.LayerNo...,<class 'torch.optim.lr_scheduler.MultiStepLR'>,0.371870
90,<function relu at 0x7efb2b1beef0>,<class 'torch.optim.adam.Adam'>,<function xavier_uniform_ at 0x7efb1d13f130>,0.0,True,<class 'torch.nn.modules.normalization.LayerNo...,<class 'torch.optim.lr_scheduler.ExponentialLR'>,0.371870
91,<function relu at 0x7efb2b1beef0>,<class 'torch.optim.adam.Adam'>,<function xavier_uniform_ at 0x7efb1d13f130>,0.0,True,<class 'torch.nn.modules.normalization.LayerNo...,<class 'torch.optim.lr_scheduler.MultiStepLR'>,0.371870


In [ ]:
pretrained_ft_res= train(all_combinations, pretrained_ft_X, y)

In [ ]:
pretrained_ft_res_list = []

In [ ]:
pretrained_ft_res.sort_values(by='F1', ascending=False)

,Activation,Optimizer,Weight Initialization,Dropout,Regularization,Normalization,Scheduler,F1
121,<function relu at 0x7efb2b1beef0>,<class 'torch.optim.sgd.SGD'>,<function xavier_uniform_ at 0x7efb1d13f130>,0.0,True,<class 'torch.nn.modules.batchnorm.BatchNorm1d'>,<class 'torch.optim.lr_scheduler.MultiStepLR'>,0.638177
105,<function relu at 0x7efb2b1beef0>,<class 'torch.optim.sgd.SGD'>,<function kaiming_uniform_ at 0x7efb1d13f2e0>,0.0,True,<class 'torch.nn.modules.batchnorm.BatchNorm1d'>,<class 'torch.optim.lr_scheduler.MultiStepLR'>,0.637744
57,<function leaky_relu at 0x7efb2b1bf2e0>,<class 'torch.optim.sgd.SGD'>,<function xavier_uniform_ at 0x7efb1d13f130>,0.0,True,<class 'torch.nn.modules.batchnorm.BatchNorm1d'>,<class 'torch.optim.lr_scheduler.MultiStepLR'>,0.637641
93,<function relu at 0x7efb2b1beef0>,<class 'torch.optim.adam.Adam'>,<function xavier_uniform_ at 0x7efb1d13f130>,0.0,False,<class 'torch.nn.modules.batchnorm.BatchNorm1d'>,<class 'torch.optim.lr_scheduler.MultiStepLR'>,0.635896
77,<function relu at 0x7efb2b1beef0>,<class 'torch.optim.adam.Adam'>,<function kaiming_uniform_ at 0x7efb1d13f2e0>,0.0,False,<class 'torch.nn.modules.batchnorm.BatchNorm1d'>,<class 'torch.optim.lr_scheduler.MultiStepLR'>,0.635062
...,...,...,...,...,...,...,...,...
42,<function leaky_relu at 0x7efb2b1bf2e0>,<class 'torch.optim.sgd.SGD'>,<function kaiming_uniform_ at 0x7efb1d13f2e0>,0.0,True,<class 'torch.nn.modules.normalization.LayerNo...,<class 'torch.optim.lr_scheduler.ExponentialLR'>,0.371870
95,<function relu at 0x7efb2b1beef0>,<class 'torch.optim.adam.Adam'>,<function xavier_uniform_ at 0x7efb1d13f130>,0.0,False,<class 'torch.nn.modules.normalization.LayerNo...,<class 'torch.optim.lr_scheduler.MultiStepLR'>,0.371870
94,<function relu at 0x7efb2b1beef0>,<class 'torch.optim.adam.Adam'>,<function xavier_uniform_ at 0x7efb1d13f130>,0.0,False,<class 'torch.nn.modules.normalization.LayerNo...,<class 'torch.optim.lr_scheduler.ExponentialLR'>,0.371870
43,<function leaky_relu at 0x7efb2b1bf2e0>,<class 'torch.optim.sgd.SGD'>,<function kaiming_uniform_ at 0x7efb1d13f2e0>,0.0,True,<class 'torch.nn.modules.normalization.LayerNo...,<class 'torch.optim.lr_scheduler.MultiStepLR'>,0.371870
